# Testing S-57 Data Processing Classes

# S-57 Data Conversion Workflows

This notebook demonstrates comprehensive S-57 (Electronic Navigational Chart) conversion workflows
using the `nautical_graph_toolkit` library. Learn how to convert raw S-57 files to multiple backend formats
and verify data integrity across the conversion pipeline.

## Conversion Modes

### 1. **by_enc Mode** (S57Base)
- Each S-57 file becomes a separate output database/schema
- Useful for: Managing individual chart versions, isolated testing, source tracking
- Output structure: One schema per ENC (e.g., `US1EEZ1M`, `US1GC09M`)

### 2. **by_layer Mode** (S57Advanced) - RECOMMENDED
- All S-57 files merged into a single database organized by object class (layer)
- Useful for: Unified analysis, spatial queries, performance optimization
- Features are stamped with `dsid_*` columns tracking source ENC, edition, and update
- Better for: Routing, maritime analysis, data fusion

## Backend Targets
- **PostGIS**: Multi-user, server-based, enterprise-ready
- **GeoPackage**: Portable single file, works offline
- **SpatiaLite**: Self-contained SQLite, no external dependencies

## Performance Considerations
- S57Advanced with `auto_tune_batch_size=True`: Automatically optimizes memory and I/O
- Parallel processing: Available for read-only file discovery (not database writes)
- Large datasets (10000+ ENCs): May take 2-4 hours depending on hardware


## Notebook Structure and Workflow

This notebook is organized into the following steps:

| Step | Name | Purpose | Duration |
|------|------|---------|----------|
| 1 | **Configuration** | Set backend, methods, and user parameters | 2 minutes |
| 2 | **Imports** | Load all dependencies and set environment | < 1 minute |
| 3 | **PostGIS Connection Test** | Verify database connectivity (if using PostGIS) | 1-2 seconds |
| 4 | **S57Base Conversions** | Quick one-to-one conversion (5-30 minutes) | 5-30 minutes |
| 5 | **S57Advanced Conversions** | Production layer-centric conversion (2-8 hours) | 2-8 hours |
| 6 | **S57Updater** | Incremental updates to maintain databases | 10-30 minutes |
| 7 | **DeepTest** | Comprehensive integration testing | 2-6 hours |

### Simplified Configuration System

Instead of managing 14 individual workflow flags, the new system uses:

**Backend Selection:**
```python
backend = 'postgis'  # Options: 'postgis', 'spatialite', 'gpkg'
```

**Conversion Methods:**
```python
methods = {
    'S57_Base': True,          # Run S57Base conversions
    'S57_Advanced': True,      # Run S57Advanced conversions (RECOMMENDED)
    'S57_Updater': False       # Run S57Updater incremental updates
}
```

**Workflow Options:**
```python
workflow_options = {
    'run_parallel_processing': False,  # PostGIS only
    'run_deeptest': False              # All backends
}
```

### How to Use This Notebook

**For Quick Testing:** 
```
backend = 'postgis'
methods = {'S57_Base': True, 'S57_Advanced': False, 'S57_Updater': False}
```
Run cells 1-4 (~5-30 minutes)

**For Production Setup:** 
```
backend = 'postgis'
methods = {'S57_Base': True, 'S57_Advanced': True, 'S57_Updater': False}
```
Run cells 1-5 (~2-8 hours first time, includes validation)

**For Updates:** 
```
backend = 'postgis'
methods = {'S57_Base': False, 'S57_Advanced': False, 'S57_Updater': True}
```
Run cells 1-3, then 6 (~10-30 minutes)

**For Validation/Testing:** 
```
backend = 'postgis'  # or 'gpkg' or 'spatialite'
workflow_options = {'run_deeptest': True}
```
Run cells 1-3, then 7 (~2-6 hours)

### Backend Notes

- **PostGIS**: All features supported (conversions, updates, parallel processing)
- **GeoPackage**: Conversions supported; updates not recommended (concurrent access issues)
- **SpatiaLite**: Conversions supported; updates not recommended (concurrent access issues)

### Important Pre-requisites

**For PostGIS Backend:**
- `.env` file must contain: `DB_NAME`, `DB_USER`, `DB_PASSWORD`, `DB_HOST`, `DB_PORT`
- Connection test (Step 3) will verify credentials

**Data Files:**
- S-57 ENC files in `data_paths['s57_data_dir']` (provided in test dataset)
- For S57Advanced, can optionally use larger dataset from `data_paths['active_enc']`

**Disk Space Requirements (FIX 6):**
- **S57Base conversions**: ~2-5x size of source .000 files
- **S57Advanced conversions**: ~3-8x size of source data (varies by backend and dataset size)
- **DeepTest**: Additional 2-3x for test outputs
- **Recommended minimum**: 10GB free space for test datasets, 100GB+ for full NOAA ENC coverage
- **SpatiaLite**: Generally smaller than GeoPackage, fastest conversion time (~1 hour for NOAA data)
- **GeoPackage**: Moderate size, good for portable single-file deployments (~4 hours for NOAA data)
- **PostGIS**: No local file size limits, distributed storage across database

## Step 1: Configuration

All user-configurable parameters are centralized below for easy modification. Adjust parameters in the next cell before running the notebook.

In [ ]:
# =============================================================================
# NOTEBOOK CONFIGURATION - Adjust these parameters before running
# =============================================================================

# --- Backend Selection (choose one) ---
# Options: 'postgis', 'spatialite', 'gpkg'
backend = 'gpkg'

# --- Conversion Methods ---
# Select which S-57 conversion methods to run
methods = {
    'S57_Base': True,       # Quick one-to-one conversion (5-30 minutes)
    'S57_Advanced': True,   # Production layer-centric conversion (2-8 hours) - RECOMMENDED
    'S57_Updater': True    # Incremental updates to existing database (10-30 minutes)
}

# --- Additional Workflow Options ---
# PostGIS-only features and integration testing
workflow_options = {
    'run_parallel_processing': False,  # PostGIS only - test parallel file discovery
    'run_deeptest': False              # Comprehensive integration testing (2-6 hours)
}

# --- Output Configuration ---
# Base name for all S57Advanced and S57Updater outputs (schemas and filenames)
# Change this single variable to rename all related outputs
output_base_name = 'enc_west'

# Derived output configuration (automatically generated from base name)
output_config = {
    # S57Base output (by_enc mode) - one database per ENC
    'base_output_dir': 'by_enc',
    
    # S57Advanced configurations (all use output_base_name)
    'pg_schema': output_base_name,
    'gpkg_filename': f'{output_base_name}.gpkg',
    'sqlite_filename': f'{output_base_name}.sqlite',
    
    # Parallel processing test configuration (separate schema for testing)
    'parallel_schema': 's57_parallel_test',
    
    # S57Updater configuration (uses same base name)
    'updater_schema': output_base_name,
    'updater_encs': ['US3CA52M', 'US1GC09M'],  # Specific ENCs to update
    'updater_gpkg': f'{output_base_name}.gpkg',
    'updater_sqlite': f'{output_base_name}.sqlite',
    
    # DeepTest integration testing (separate schema for testing)
    'deeptest_schema': 's57_deeptest',
}

# --- S57Updater Configuration ---
# Control update behavior when methods['S57_Updater'] is enabled
updater_config = {
    'force_update': True,  # True: Replace ENCs entirely | False: Apply incremental updates
}

# --- DeepTest Configuration ---
# Settings for comprehensive integration testing
deeptest_config = {
    'test_level': 3,                     # Test depth (1=minimal, 3=comprehensive)
    'skip_postgis': False,               # Skip PostGIS tests
    'skip_updates': False,               # Skip update testing
    'cleanup_on_success': True,          # Clean up test artifacts after success
    'clean_output': True,                # Clean output directory before test
    'exclude_geometry_cols': ["geometry", "geom", "wkb_geometry"]  # Columns to exclude from output
}

# --- Configuration Summary ---
print("=" * 70)
print("✓ Configuration loaded successfully!")
print("=" * 70)
print(f"\n📦 Backend: {backend.upper()}")
print(f"📊 Methods:")
print(f"   - S57_Base: {methods['S57_Base']}")
print(f"   - S57_Advanced: {methods['S57_Advanced']}")
print(f"   - S57_Updater: {methods['S57_Updater']}")
if backend == 'postgis':
    print(f"   - Parallel Processing: {workflow_options['run_parallel_processing']}")
print(f"   - DeepTest: {workflow_options['run_deeptest']}")
if methods['S57_Updater']:
    update_mode = "Force update (replace ENCs)" if updater_config['force_update'] else "Normal update (incremental)"
    print(f"\n🔄 S57Updater mode: {update_mode}")
print(f"\n📁 Output base name: {output_base_name}")
print(f"   - S57Advanced schema/file: {output_config['pg_schema']}")
print(f"   - S57Updater schema/file: {output_config['updater_schema']}")
print("=" * 70)
print("\n⚠️  Paths and database parameters will be set after STEP 2: IMPORTS")
print("   (These depend on project_root and environment variables)")
print("=" * 70)

In [ ]:
# =============================================================================
# STEP 2: IMPORTS - Consolidated import statements for all notebook dependencies
# =============================================================================

# --- Standard Library ---
import sys
import os
import logging
from pathlib import Path

# --- Environment Setup ---
from dotenv import load_dotenv

# --- Geospatial Libraries ---
from osgeo import gdal
import geopandas as gpd
import fiona

# --- Jupyter/Display ---
from IPython.display import display

# --- Environment Initialization ---
# (These run during import phase to ensure configuration is available early)
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# --- S-57 Data Processing Core Classes ---
from src.nautical_graph_toolkit.core.s57_data import (
    S57Base,
    S57Advanced,
    S57Updater,
    PostGISManager,
    SpatiaLiteManager,
    GPKGManager,
    S57AdvancedConfig
)

# --- Database Connectivity ---
from src.nautical_graph_toolkit.utils.db_utils import PostGISConnector

# Load environment variables (.env file should be at project root)
load_dotenv(project_root / ".env")

# --- Conditional Imports for DeepTest (only if needed) ---
# DeepTest imports are loaded later in workflow_steps['run_deeptest'] section
# This avoids importing test dependencies unless specifically requested

print("✓ All imports loaded successfully!")
print(f"✓ Python path includes: {project_root}")
print(f"✓ GDAL version: {gdal.__version__}")
print(f"✓ GeoPandas version: {gpd.__version__}")

## Step 2.5: Data Paths and Database Setup

Now that imports are complete, set up data paths and database connection parameters.

In [ ]:
# --- Data Source Paths ---
# Define locations for input ENC data and output databases
data_paths = {
    # Primary ENC data directory (used for S57Advanced large-scale conversions)
    # Contains complete NOAA ENC dataset for comprehensive testing
    'active_enc': project_root / 'data' / 'ENC_SF_LA' / 'ENC_ROOT',
    
    # Test/sample ENC data directory (used for S57Base and smaller tests)
    # Contains subset of ENCs for quick conversions and verification
    's57_data_dir': project_root / 'data' / 'ENC_ROOT',
    
    # ENC update files (used by S57Updater for incremental updates)
    # Contains newer chart updates to test update mechanisms
    's57_data_update_dir': project_root / 'data' / 'ENC_ROOT_UPDATE',
    
    # Output directory for all conversion results
    'output_dir': Path.cwd() / 'output',
    
    # Test output directory (used by DeepTest integration testing)
    'test_output_dir': project_root / 'test_output'
}

# FIX 5: Validate critical paths exist before proceeding
# This prevents silent failures if data directories are missing
print("Validating data paths...")
required_paths = {
    's57_data_dir': data_paths['s57_data_dir'],
    'output_dir': data_paths['output_dir']
}

for path_name, path in required_paths.items():
    if path.exists():
        print(f"  ✓ {path_name}: {path}")
    else:
        print(f"  ⚠️  {path_name} does not exist: {path}")
        print(f"     Create this directory or adjust data_paths configuration.")

# --- Database Connection Parameters ---
# PostGIS connection details (from .env file)
# Only used when backend = 'postgis'
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

# --- S57Advanced Configuration Objects ---
# Define conversion behavior for different scenarios
s57_advanced_config = {
    # Default configuration: auto-tuned batch processing
    # Recommended for most conversions - balances speed and memory
    'default': S57AdvancedConfig(
        auto_tune_batch_size=True,
        enable_debug_logging=False
    ),

    # Parallel processing configuration: high-safety read-only approach
    # Safe for production - only parallelizes file discovery, not database writes
    'parallel': S57AdvancedConfig(
        enable_parallel_processing=True,    # Use multiple cores
        parallel_read_only=True,            # Read-only operations only
        parallel_db_writes=False,           # Writes are sequential
        parallel_validation_level='strict', # Maximum data quality checks
        max_parallel_workers=2,             # Conservative worker count for stability
        enable_debug_logging=False          # Disable debug logging during config creation
    )
}

# --- Summary ---
print("=" * 70)
print("✓ Paths and Database Parameters Configured!")
print("=" * 70)
print(f"\n📁 Data directories:")
print(f"   - Active ENC: {data_paths['active_enc']}")
print(f"   - Test ENC:   {data_paths['s57_data_dir']}")
print(f"   - Output:     {data_paths['output_dir']}")
if backend == 'postgis':
    print(f"\n🗄️  PostGIS Connection:")
    print(f"   - Database: {db_params.get('dbname', 'NOT CONFIGURED')}")
    print(f"   - Host:     {db_params.get('host', 'NOT CONFIGURED')}")
print("=" * 70)

In [ ]:
# =============================================================================
# STEP 3: PostGIS Connection Test
# =============================================================================
# IMPORTANT: Test the database connection early for PostGIS backend
# This ensures all PostGIS operations will succeed before proceeding.

if backend == 'postgis':
    print("=" * 70)
    print("Testing PostGIS database connection...")
    print("=" * 70)
    try:
        pg = PostGISConnector(db_params)
        pg.connect()
        schemas = pg.get_schemas()
        print(f"✓ PostGIS connection successful!")
        print(f"✓ Available schemas: {schemas}")
        print("=" * 70)
    except Exception as e:
        print(f"✗ PostGIS connection failed: {e}")
        print("  Please check your .env file and database credentials before proceeding.")
        print("=" * 70)
else:
    print("=" * 70)
    print(f"PostGIS connection test skipped (backend = '{backend}', not 'postgis')")
    print("=" * 70)

In [ ]:
# Check GDAL version (always runs - no flag to skip)
print("=" * 70)
print("GDAL and Library Versions:")
print("=" * 70)
print(f"GDAL Python bindings version: {gdal.__version__}")
print(f"GDAL C library version: {gdal.VersionInfo('RELEASE_NAME')}")
print(f"GeoPandas version: {gpd.__version__}")
print(f"Fiona version: {fiona.__version__}")
print("=" * 70)

## Step 4: S57Base - Simple Bulk Conversion (by_enc mode)

`S57Base` is the simplest conversion class, creating one output per input S-57 file.

### Key Characteristics

| Feature | Details |
|---------|---------|
| **Processing Mode** | One S-57 file → One output database/schema |
| **Tool Used** | GDAL's `VectorTranslate` |
| **Feature Stamping** | None - raw ENC data only |
| **Speed** | Fast - minutes to hours depending on ENC count |
| **Output Locations** | Multiple backends: GeoPackage, PostGIS, SpatiaLite |

### Use Cases
- Initial ENC assessment and validation
- Isolated testing of specific charts
- Preserving individual ENC versions unchanged
- Quick conversions of small datasets
- Debugging specific ENC problems

### What Happens During Conversion
1. Scans input directory for `.000` files (S-57 base files)
2. Uses GDAL's `VectorTranslate` for one-to-one mapping
3. Creates separate output per chart
4. Preserves original S-57 object class organization

### Limitation
No feature stamping or cross-chart analysis capability - see S57Advanced for production use.

## Comparison: S57Base vs S57Advanced

| Aspect | S57Base (by_enc) | S57Advanced (by_layer) |
|--------|------------------|----------------------|
| **Output Structure** | One database per ENC file | Single unified database with all ENCs |
| **Organization** | By source chart | By S-57 object class (layer) |
| **Feature Stamping** | None | Yes: `dsid_dsnm`, `dsid_edtn`, `dsid_updn` |
| **Cross-chart Analysis** | Limited | Excellent - all layers contain all relevant features |
| **Conversion Speed** | Fast (minutes) | Slow (hours) - but only needed once |
| **Best For** | Testing, QA, isolated analysis | Production integration, spatial queries, routing |
| **Data Duplication** | High - object classes repeated in each ENC | None - normalized by layer |
| **Spatial Queries** | Must query multiple databases | Single database query |

### When to Use Each Mode

**Use S57Base if you need to:**
- Quickly test a few ENC files
- Preserve original ENC boundaries and organization
- Work with individual chart editions
- Perform isolated validation checks

**Use S57Advanced if you need to:**
- Build a production maritime database
- Run spatial analysis across chart boundaries
- Support routing and pathfinding algorithms
- Track data provenance with `dsid_*` columns
- Minimize storage overhead

#### Convert to GPKG

In [ ]:
# S57Base conversion to selected backend
if methods['S57_Base'] and backend == 'gpkg':
    # import logging
    # logging.getLogger('src.nautical_graph_toolkit.core.s57_data').setLevel(logging.DEBUG)

    base_converter = S57Base(
            input_path=data_paths['s57_data_dir'],
            output_dest=str(data_paths['output_dir'] / 'by_enc'),
            output_format='gpkg',
            overwrite=True
        )
    base_converter.convert_by_enc()
elif methods['S57_Base']:
    print(f"S57Base GeoPackage conversion skipped (backend = '{backend}', not 'gpkg')")
else:
    print("S57Base conversion skipped (methods['S57_Base'] = False)")

#### Convert to PostGIS

In [ ]:
# S57Base conversion to PostGIS
if methods['S57_Base'] and backend == 'postgis':
    base_converter = S57Base(
            input_path=data_paths['s57_data_dir'],
            output_dest=db_params,
            output_format='postgis',
            overwrite=False
        )
    base_converter.convert_by_enc()
elif methods['S57_Base']:
    print(f"S57Base PostGIS conversion skipped (backend = '{backend}', not 'postgis')")
else:
    print("S57Base conversion skipped (methods['S57_Base'] = False)")

In [ ]:
# Verify PostGIS schema after S57Base conversion
if methods['S57_Base'] and backend == 'postgis':
    pg = PostGISConnector(db_params)
    pg.connect()
    pg.get_schema_summary()
elif methods['S57_Base']:
    print(f"PostGIS schema verification skipped (backend = '{backend}', not 'postgis')")
else:
    print("PostGIS schema verification skipped (S57Base conversion not enabled)")

#### Convert to SpatiaLite

In [ ]:
# S57Base conversion to SpatiaLite
if methods['S57_Base'] and backend == 'spatialite':
    base_converter = S57Base(
            input_path=data_paths['s57_data_dir'],
             output_dest=str(data_paths['output_dir'] / 'by_enc'),
            output_format='spatialite',
            overwrite=True
        )
    base_converter.convert_by_enc()
elif methods['S57_Base']:
    print(f"S57Base SpatiaLite conversion skipped (backend = '{backend}', not 'spatialite')")
else:
    print("S57Base conversion skipped (methods['S57_Base'] = False)")

## Step 5: S57Advanced - Optimized Layer-Centric Conversion (by_layer mode) - RECOMMENDED

`S57Advanced` performs optimized, layer-centric conversion merging all inputs into a unified database. **This is the recommended approach for production maritime databases.**

### Key Features

| Feature | Benefit |
|---------|---------|
| **Layer-centric organization** | All data organized by S-57 object class (e.g., all soundings together) - enables powerful spatial queries |
| **Feature stamping** | Each feature tagged with source ENC (`dsid_dsnm`), edition (`dsid_edtn`), update (`dsid_updn`) - full data lineage |
| **Batch processing** | Optimized memory management with `auto_tune_batch_size` - handles 10000+ ENCs |
| **Parallel processing** | Optional multi-threaded file discovery (read-only) - faster preprocessing |
| **Auto-tuning** | Automatically adjusts batch size based on available memory - no manual tuning needed |

### Use Cases
- **Production data integration**: Build comprehensive maritime databases supporting multi-user access
- **Spatial analysis**: Query all features of a type across all charts (e.g., "all lights in US waters")
- **Routing and pathfinding**: Need unified navigable area representation for navigation algorithms
- **Data fusion**: Combine S-57 with other maritime data sources
- **Historical tracking**: Use `dsid_edtn` and `dsid_updn` columns to analyze chart evolution

### Configuration Options
- `auto_tune_batch_size`: Let system optimize performance automatically (recommended)
- `enable_parallel_processing`: Use multiple cores for file discovery
- `parallel_validation_level`: Balance speed vs. data quality checks ('strict' is recommended)

### Why Layer-Centric is Better for Production
1. **Single Query**: One SQL query gets all relevant features instead of querying 100+ databases
2. **Data Integrity**: Feature stamping ensures traceability of data sources
3. **Performance**: Spatial indexes optimized for layer-based queries
4. **Consistency**: All features in a layer have uniform attributes and structure

In [ ]:
# S57Advanced conversion to PostGIS
if methods['S57_Advanced'] and backend == 'postgis':
    # --- Step 1: Conversion ---
    # NOAA ENC conversion - 2h 24m
    pg_schema = output_config['pg_schema']
    print(f"--- Starting S57Advanced conversion to PostGIS schema: '{pg_schema}' ---")

    advanced_converter_pg = S57Advanced(
        input_path=data_paths['active_enc'],
        output_dest=db_params,
        output_format='postgis',
        overwrite=True,
        schema=pg_schema,
        config=s57_advanced_config['default']
    )
    advanced_converter_pg.convert_to_layers()

    print("\n--- Step 2: Verification ---")
    # --- Step 2: Verification ---
    try:
        manager = PostGISManager(db_params=db_params, schema=pg_schema)

        # Verify a common layer exists and has data
        depare_layer = manager.get_layer('lndmrk')
        print(f"Loaded {len(depare_layer)} features from LNDMRK layer.")
        print("LNDMRK layer head (note the 'dsid_*' stamping columns):")
        display(depare_layer.head())

        # Verify feature stamping integrity across all layers
        print("\nVerifying feature update status against DSID table...")
        verification_results = manager.verify_feature_update_status()
        print("Verification Summary:")
        display(verification_results)

    except Exception as e:
        print(f"An error occurred during PostGIS verification: {e}")
elif methods['S57_Advanced']:
    print(f"S57Advanced PostGIS conversion skipped (backend = '{backend}', not 'postgis')")
else:
    print("S57Advanced conversion skipped (methods['S57_Advanced'] = False)")

### 3.2. Convert to GeoPackage and Verify

Next, we test the conversion to a single GeoPackage file. The `GPKGManager` is used for verification.

In [ ]:
# S57Advanced conversion to GeoPackage
if methods['S57_Advanced'] and backend == 'gpkg':
    # --- Step 1: Conversion ---
    # NOAA ENC conversion - 4h 9m
    gpkg_path = data_paths['output_dir'] / output_config['gpkg_filename']
    print(f"--- Starting S57Advanced conversion to GeoPackage: '{gpkg_path}' ---")

    advanced_converter_gpkg = S57Advanced(
        input_path=data_paths['active_enc'],
        output_dest=str(gpkg_path),
        output_format='gpkg',
        overwrite=True,
        config=s57_advanced_config['default']
    )
    advanced_converter_gpkg.convert_to_layers()

    print("\n--- Step 2: Verification ---")
    # --- Step 2: Verification ---
    try:
        manager = GPKGManager(gpkg_path=gpkg_path)

        # Verify a common layer exists and has data
        soundg_layer = manager.get_layer('lndmrk')
        print(f"Loaded {len(soundg_layer)} features from LNDMRK layer in the GeoPackage.")
        print("LNDMRK layer head:")
        display(soundg_layer.head())

        # Verify feature stamping integrity
        print("\nVerifying feature update status...")
        verification_results = manager.verify_feature_update_status()
        print("Verification Summary:")
        display(verification_results)

    except Exception as e:
        print(f"An error occurred during GeoPackage verification: {e}")
elif methods['S57_Advanced']:
    print(f"S57Advanced GeoPackage conversion skipped (backend = '{backend}', not 'gpkg')")
else:
    print("S57Advanced conversion skipped (methods['S57_Advanced'] = False)")

### 3.3. Convert to SpatiaLite and Verify

Finally, we test the conversion to a SpatiaLite database file and verify it with `SpatiaLiteManager`.

In [ ]:
# S57Advanced conversion to SpatiaLite
if methods['S57_Advanced'] and backend == 'spatialite':
    # --- Step 1: Conversion ---
    # NOAA ENC conversion - 1h 1m
    sqlite_path = data_paths['output_dir'] / output_config['sqlite_filename']
    print(f"--- Starting S57Advanced conversion to SpatiaLite: '{sqlite_path}' ---")

    advanced_converter_sqlite = S57Advanced(
        input_path=data_paths['active_enc'],
        output_dest=str(sqlite_path),
        output_format='spatialite',
        overwrite=True,
        config=s57_advanced_config['default']
    )
    advanced_converter_sqlite.convert_to_layers()

    print("\n--- Step 2: Verification ---")
    # --- Step 2: Verification ---
    try:
        manager = SpatiaLiteManager(db_path=sqlite_path)

        # Verify a common layer exists and has data
        boyspp_layer = manager.get_layer('boyspp')
        print(f"Loaded {len(boyspp_layer)} features from BOYSPP layer in the SpatiaLite DB.")
        print("BOYSPP layer head:")
        display(boyspp_layer.head())

        # Verify feature stamping integrity
        print("\nVerifying feature update status...")
        verification_results = manager.verify_feature_update_status()
        print("Verification Summary:")
        display(verification_results)

    except Exception as e:
        print(f"An error occurred during SpatiaLite verification: {e}")
elif methods['S57_Advanced']:
    print(f"S57Advanced SpatiaLite conversion skipped (backend = '{backend}', not 'spatialite')")
else:
    print("S57Advanced conversion skipped (methods['S57_Advanced'] = False)")

### 3.4. Test with Enterprise-Safe Parallel Processing

This test demonstrates using the `S57AdvancedConfig` to enable high-safety, read-only parallel processing, which can speed up the initial file discovery and preprocessing steps.

In [ ]:
# Test S57Advanced with Enterprise-Safe Parallel Processing (PostGIS only)
if workflow_options['run_parallel_processing'] and backend == 'postgis':
    # --- Step 1: Configure and Convert ---
    parallel_schema = output_config['parallel_schema']
    print(f"--- Starting S57Advanced conversion with PARALLEL processing to schema: '{parallel_schema}' ---")

    # Use the recommended high-safety configuration for parallel reads
    high_safety_config = s57_advanced_config['parallel']

    print("\nUsing configuration:")
    print(high_safety_config.get_configuration_summary())

    advanced_converter_parallel = S57Advanced(
        input_path=data_paths['s57_data_dir'],
        output_dest=db_params,
        output_format='postgis',
        overwrite=True,
        schema=parallel_schema,
        config=high_safety_config
    )
    advanced_converter_parallel.convert_to_layers()

    print("\n--- Step 2: Verification ---")
    # --- Step 2: Verification ---
    try:
        manager = PostGISManager(db_params=db_params, schema=parallel_schema)
        summary = manager.get_enc_summary()
        print(f"Successfully created {len(summary)} ENCs in the parallel-processed schema.")
        display(summary.head())
    except Exception as e:
        print(f"An error occurred during parallel processing verification: {e}")
elif workflow_options['run_parallel_processing']:
    print(f"Parallel processing test skipped (backend = '{backend}', not 'postgis')")
    print("Parallel processing is only available with PostGIS backend.")
else:
    print("Parallel processing test skipped (workflow_options['run_parallel_processing'] = False)")

## Step 6: S57Updater - Incremental Updates

`S57Updater` handles incremental updates to an existing S57Advanced database, maintaining a production ENC database with latest chart updates.

### Update Modes

| Mode | Method | Use Case | Configuration |
|------|--------|----------|---|
| **Force Update** | `force_update_from_location()` | Replace specific ENCs entirely (e.g., after fixing errors) | `updater_config['force_update'] = True` |
| **Normal Update** | `update_from_location()` | Apply incremental updates if available from NOAA | `updater_config['force_update'] = False` |

**Configure which mode to use in Step 1:**
```python
updater_config = {
    'force_update': True,  # True: Replace ENCs entirely | False: Apply incremental updates
}
```

### How It Works
1. Compares local ENC versions with update files or source data
2. Re-converts and replaces features **only for changed ENCs** (fast!)
3. Maintains transaction integrity - all changes succeed or all fail (ACID)
4. Provides detailed change summaries showing before/after versions

### Change Summary Information
After an update, you can see:
- Old Edition → New Edition for each updated ENC
- Update number changes tracking version history
- All changes recorded in database metadata tables

### Backend Support

| Backend | Status | Notes |
|---------|--------|-------|
| **PostGIS** | ✅ Fully Supported | Transactional updates with full ACID guarantees |
| **SpatiaLite** | ⚠️ Use with Care | Works but can cause "database disk image is malformed" errors with concurrent access |
| **GeoPackage** | ⚠️ Use with Care | Similar issues to SpatiaLite with concurrent access |

### Important Caution
Only use with PostGIS for production systems. File-based updates (SpatiaLite/GeoPackage) can cause corruption if the file is accessed by multiple uncoordinated connections simultaneously. **PostGIS is the only backend recommended for update operations in production.**

### Example Update Summary Output (FIX 8)

When running S57Updater with `force_update_from_location()`, you'll see output like this showing edition and update number changes for each ENC:

| Old Edition | New Edition |
|--|--|
| US1EEZ1M 9:0 | 9:0 |
| US1GC09M 65:5 | 71:1 |
| US1PO02M 21:1 | 21:1 |
| US2WC12M 27:8 | 27:12 |
| US3CA52M 31:0 | 31:7 |
| US4CA60M 37:6 | 38:2 |

The format is `ENC_NAME edition:update`. This example shows several ENCs were updated to newer editions/updates, while others (US1EEZ1M, US1PO02M) were already current.

### PostGIS Updater

In [ ]:
# Test S57Updater with selected backend
if methods['S57_Updater'] and backend == 'postgis':
    db_schema = output_config['updater_schema']
    updater = S57Updater(output_format='postgis',
                         dest_conn=db_params,
                         schema=db_schema)
    
    # Execute selected update mode based on configuration
    if updater_config['force_update']:
        print("Running force update (replacing specific ENCs)...")
        updater.force_update_from_location(
            data_paths['s57_data_dir'], 
            enc_filter=output_config['updater_encs']
        )
    else:
        print("Running normal incremental update...")
        updater.update_from_location(data_paths['s57_data_update_dir'])
    
    summary_df = updater.get_change_summary()
    display(summary_df)
elif methods['S57_Updater']:
    print(f"S57Updater PostGIS test skipped (backend = '{backend}', not 'postgis')")
else:
    print("S57Updater test skipped (methods['S57_Updater'] = False)")

### 4.2. SpatiaLite Updater

This test demonstrates the `S57Updater` functionality for a SpatiaLite database. The previous implementation failed with a `database disk image is malformed` error because it attempted to run the updater on a file that was being accessed by multiple, uncoordinated connections (OGR and SQLAlchemy).

The corrected test follows a robust, isolated process:

1.  **Create a fresh SpatiaLite database** from the initial ENC data (`s57_data_dir`). This provides a clean, known state to update and prevents file corruption.
2.  **Run the updater** on this new database using the update data (`s57_data_update_dir`).
3.  **Verify the results** by comparing the ENC versions before and after the update and checking for data integrity.

In [ ]:
# Test S57Updater with SpatiaLite
if methods['S57_Updater'] and backend == 'spatialite':
    print("⚠️  WARNING: SpatiaLite updates can cause 'database disk image is malformed' errors")
    print("   PostGIS is recommended for production update operations.")
    print()
    
    sqlite_update_target_path = data_paths['output_dir'] / output_config['updater_sqlite']
    updater = S57Updater(
         output_format='spatialite',
         dest_conn=str(sqlite_update_target_path),
         schema='main' # SpatiaLite doesn't use schemas like PostGIS, but 'main' is the default
    )
    
    # Execute selected update mode based on configuration
    if updater_config['force_update']:
        print("Running force update (replacing specific ENCs)...")
        updater.force_update_from_location(
            data_paths['s57_data_dir'], 
            enc_filter=output_config['updater_encs']
        )
    else:
        print("Running normal incremental update...")
        updater.update_from_location(data_paths['s57_data_update_dir'])
    
    summary_df = updater.get_change_summary()
elif methods['S57_Updater']:
    print(f"S57Updater SpatiaLite test skipped (backend = '{backend}', not 'spatialite')")
else:
    print("S57Updater test skipped (methods['S57_Updater'] = False)")

### GPKG Updater

In [ ]:
# Test S57Updater with GeoPackage
if methods['S57_Updater'] and backend == 'gpkg':
    print("⚠️  WARNING: GeoPackage updates can cause corruption with concurrent access")
    print("   PostGIS is recommended for production update operations.")
    print()
    
    update_target_path = data_paths['output_dir'] / output_config['updater_gpkg']
    updater = S57Updater(
         output_format='gpkg',
         dest_conn=str(update_target_path),
         schema='main' # GeoPackage doesn't use schemas like PostGIS, but 'main' is the default
    )
    
    # Execute selected update mode based on configuration
    if updater_config['force_update']:
        print("Running force update (replacing specific ENCs)...")
        updater.force_update_from_location(
            data_paths['s57_data_dir'], 
            enc_filter=output_config['updater_encs']
        )
    else:
        print("Running normal incremental update...")
        updater.update_from_location(data_paths['s57_data_update_dir'])
    
    summary_df = updater.get_change_summary()
elif methods['S57_Updater']:
    print(f"S57Updater GeoPackage test skipped (backend = '{backend}', not 'gpkg')")
else:
    print("S57Updater test skipped (methods['S57_Updater'] = False)")

## Step 7: DeepTest - Comprehensive Integration Testing

DeepTest is a comprehensive integration testing framework that exercises the complete S-57 conversion pipeline across all backends and scenarios.

### What DeepTest Tests

| Component | Tests | Duration |
|-----------|-------|----------|
| **S57Base Conversions** | All three backends (PostGIS, GeoPackage, SpatiaLite) | ~1 hour |
| **S57Advanced Conversions** | All three backends with full dataset | ~2-8 hours |
| **Update Mechanisms** | Apply updates to each backend independently | ~30 minutes |
| **Data Integrity** | Feature counts, geometry validity, dsid stamping | Continuous |
| **Multi-user Access** | Concurrent database connections | Depends on backend |

### Test Levels

| Level | Scope | Duration | When to Use |
|-------|-------|----------|------------|
| **1 (Minimal)** | Quick smoke test on small ENC subset | ~15 minutes | CI/CD pipelines, quick validation |
| **2 (Standard)** | Full S57Base and S57Advanced conversions | ~2 hours | Regular testing, code validation |
| **3 (Comprehensive)** | All conversions + updates + multi-user testing | ~6 hours | Release testing, certification |

### Configuration Options in DeepTest

| Option | Purpose | Default |
|--------|---------|---------|
| `test_level` | How thorough the test should be (1-3) | 3 |
| `skip_postgis` | Skip PostGIS tests | False |
| `skip_updates` | Skip update mechanism tests | False |
| `cleanup_on_success` | Delete test artifacts after successful run | True |
| `clean_output` | Clear output directory before starting | True |

### Why Run DeepTest
1. **Verify Conversion Pipeline**: Ensure all backends work correctly
2. **Regression Testing**: Catch breaking changes in library updates
3. **Data Quality**: Validate that converted data maintains integrity
4. **Performance Baseline**: Track conversion times across versions
5. **Certification**: Prove system ready for production deployment

#### Setup

In [ ]:
# --- Conditional DeepTest Imports ---
# Only import if we plan to run DeepTest (conditional import)
if workflow_options.get('run_deeptest', False):
    from tests.core__real_data.deep_test_s57_workflow import *
else:
    print("DeepTest imports skipped (workflow_options['run_deeptest'] = False)")
    print("To enable DeepTest, set workflow_options['run_deeptest'] = True in STEP 1 configuration")

if workflow_options.get('run_deeptest', False):
    # --- DeepTest Configuration (used only if run_deeptest=True) ---
    db_params_test = {
        'dbname': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD'),
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT')
    }
    db_schema = 's57_deeptest'
    test_output_path = project_root / 'test_output'

    test_config = TestConfig(s57_data_root = data_paths['s57_data_dir'],
                             s57_update_root=data_paths['s57_data_update_dir'],
                             test_output_dir = test_output_path,
                             test_level = deeptest_config['test_level'],

                             skip_postgis = deeptest_config['skip_postgis'],
                             skip_updates = deeptest_config['skip_updates'],
                             cleanup_on_success = deeptest_config['cleanup_on_success'],
                             postgis_config = db_params_test,
                             test_schema_name = db_schema,
                             clean_output = deeptest_config['clean_output'],
                             exclude_extra_cols= deeptest_config['exclude_geometry_cols'],
    )


    if workflow_options.get('run_deeptest', False):
        print(f"\n--- Starting S57 Deep Test process with TEST CONFIG ---")
        print(f"\n--- Setting up directories:")
        print(f"\nTest dataset at:       {test_config.s57_data_root}")
        print(f"\nUpdate dataset at:     {test_config.s57_update_root}")
        print(f"\nTest output directory: {test_config.test_output_dir}")
        print(f"\n-----------------------------------")
        post_conf = (f"Database: {db_params_test['dbname']} | Schema/Filename: {db_schema}") if  test_config.skip_postgis == False else ""
        print(f"\nPostGIS tests:   {'❌' if test_config.skip_postgis == True else '✅'} {post_conf}")
        print(f"\nUpdates process: {'❌' if test_config.skip_updates == True else '✅'}")

#### Load Dataset

In [ ]:
# Execute DeepTest
if workflow_options.get('run_deeptest', False):
    try:
        print(f"\n🎉 Commence DeepTest!")
        tester = S57DeepTester(test_config)
    except Exception as e:
        print(f"DeepTest execution failed: {e}")
        import traceback
        traceback.print_exc()

    # Testing Update Readiness
    if test_config.skip_updates == False:
        compare_df = tester.analyze_update_readiness()
        display(compare_df)
else:
    print("DeepTest execution skipped (workflow_options['run_deeptest'] = False)")
    print("To run DeepTest, set workflow_options['run_deeptest'] = True in STEP 1 configuration")

In [ ]:
# Run comprehensive DeepTest
if workflow_options.get('run_deeptest', False):
    try:
        report = tester.run_comprehensive_test()

        print(f"\n🎉 DeepTest completed successfully!")
        print(f"📁 Results saved to: {test_config.test_output_dir}")

    except Exception as e:
        print(f"DeepTest execution failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("Comprehensive DeepTest skipped (workflow_options['run_deeptest'] = False)")

In [ ]:
# =============================================================================
# POST-DEEPTEST VERIFICATION AND DATA EXPLORATION
# =============================================================================
# This cell validates the DeepTest workflow output by:
# 1. Opening the test-generated database (GeoPackage or SpatiaLite)
# 2. Enumerating all S-57 layers that were successfully converted
# 3. Reading and inspecting a specific layer (landmarks) to verify:
#    - Data accessibility and integrity
#    - Feature count and schema correctness
#    - Presence of feature stamping columns (dsid_*) proving S57Advanced worked
# 
# This serves as both verification that the entire S-57 conversion pipeline
# succeeded AND as a demonstration of how to programmatically access converted data.
# =============================================================================

# --- DEPENDENCIES (Already imported in STEP 2: IMPORTS) ---
# - GeoPandas: For reading spatial data from the database
# - Fiona: For layer discovery and format handling
# - project_root: Set in STEP 2, points to repository root

# =============================================================================
# SECTION 1: CONFIGURE TEST OUTPUT PATHS
# =============================================================================
# Define the location of DeepTest artifacts created in Step 7.
# DeepTest generates one of two formats depending on backend selection:

test_output_path = project_root / "tests" / "core__real_data" / "test_output"

# BACKEND CHOICE: Uncomment the format you want to inspect
# Both are identical in content, just different file formats generated by DeepTest
test_file = test_output_path / "s57_deeptest.sqlite"       # SpatiaLite format (1h 1m conversion time)
# test_file = test_output_path / "s57_deeptest.gpkg"       # GeoPackage format (4h 9m conversion time)

# RATIONALE FOR COMMENTS:
# - Both formats contain the same converted S-57 data from the same DeepTest run
# - SQLite (SpatiaLite) is currently active because it showed faster conversion
# - GeoPackage is kept as an alternative to show both backends produce identical results
# - You can uncomment either line to switch between formats

print(f"Test output directory: {test_output_path}")
print(f"Test database file:    {test_file}")

# =============================================================================
# SECTION 2: DISCOVER ALL AVAILABLE LAYERS
# =============================================================================
# The test database contains multiple layers (S-57 object classes).
# Fiona provides layer discovery without loading all data into memory.

try:
    # List all layers/tables in the spatial database
    # Returns a list of S-57 object class names (e.g., 'lndmrk', 'lndelv', 'soundg')
    layer_names = fiona.listlayers(test_file)
    
    print(f"✓ Layers found in '{test_file.name}':")
    for name in layer_names:
        print(f"  - {name}")
    
    # DATA QUALITY CHECK:
    # The presence of many layers (typically 60-80) indicates successful conversion
    # of the full S-57 dataset with all object classes represented
    
except fiona.errors.DriverError as e:
    # File access error: Database may be corrupted, locked, or inaccessible
    print(f"✗ Error: Could not open the file '{test_file}'.")
    print(f"  Please check that:")
    print(f"    1. The path is correct: {test_file}")
    print(f"    2. DeepTest has completed successfully (workflow_options['run_deeptest'] = True)")
    print(f"    3. The file is not corrupted or locked by another process")
    print(f"  Technical details: {e}")
    exit()  # Stop execution if we can't access the test data

# =============================================================================
# SECTION 3: SELECT AND READ A SPECIFIC LAYER
# =============================================================================
# Demonstrates how to programmatically extract and analyze a single S-57 layer.
# We use the LNDMRK (Landmark) layer as an example.

selected_layer_name = 'lndmrk'  # S-57 Object Class: Landmarks (lights, towers, beacons, etc.)

if selected_layer_name in layer_names:
    print(f"\n✓ Reading layer: '{selected_layer_name}'...")
    
    # Load the entire layer into a GeoDataFrame (in-memory analysis)
    # engine='fiona' ensures consistent handling of S-57 data types across formats
    gdf = gpd.read_file(test_file, layer=selected_layer_name, engine="fiona")
    
    # FEATURE STAMPING VALIDATION:
    # If S57Advanced conversion succeeded, each feature includes metadata:
    # - dsid_dsnm: Source ENC name (e.g., 'US1EEZ1M')
    # - dsid_edtn: ENC edition number (e.g., 9)
    # - dsid_updn: ENC update number (e.g., 0)
    # These columns prove the feature came from a specific chart version
    
    print(f"\n✓ Successfully loaded {len(gdf)} features from '{selected_layer_name}'.")
    print(f"  (This landmark layer contains {len(gdf)} nautical features)")
    
    # OPTIONAL DATA FILTERING:
    # Demonstrate filtering by a specific attribute
    # colpat = Color Pattern (e.g., red/white striped)
    test_df = gdf[gdf['colpat'].notna()]
    print(f"\n  Filtered subset: {len(test_df)} features with color pattern information")
    
    # =============================================================================
    # SECTION 4: INSPECT DATA SCHEMA AND QUALITY
    # =============================================================================
    # Display comprehensive schema information to verify data integrity
    
    print(f"\n✓ Layer Information and Schema:")
    print(f"  Total features: {len(gdf)}")
    print(f"  Total attributes: {len(gdf.columns)}")
    print(f"\n  Column breakdown:")
    gdf.info()  # Displays:
                # - Column names and data types
                # - Non-null counts (missing data detection)
                # - Memory usage
    
    # CRITICAL VALIDATION CHECKS:
    # 1. Presence of 'geometry' column confirms geospatial data is intact
    # 2. Presence of 'dsid_dsnm', 'dsid_edtn', 'dsid_updn' confirms feature stamping
    # 3. Non-null geometry on all features confirms successful conversion
    # 4. S-57 standard columns (rcid, objl, catlmk, etc.) indicate proper schema
    
    print(f"\n✓ Data validation passed:")
    print(f"  ✓ Geometry column present: {('geometry' in gdf.columns)}")
    print(f"  ✓ Feature stamping present: {all(col in gdf.columns for col in ['dsid_dsnm', 'dsid_edtn', 'dsid_updn'])}")
    print(f"  ✓ S-57 schema intact: {('objl' in gdf.columns and 'catlmk' in gdf.columns)}")
    
else:
    # Layer selection error: The requested layer doesn't exist in the database
    print(f"\n✗ Error: Layer '{selected_layer_name}' not found in the database.")
    print(f"\nAvailable layers:")
    for name in sorted(layer_names):
        print(f"  - {name}")
    print(f"\nTo inspect a different layer, change 'selected_layer_name' above.")

# =============================================================================
# NEXT STEPS
# =============================================================================
# Now that you've verified the DeepTest output, you can:
# 1. Modify 'selected_layer_name' to inspect other S-57 object classes
# 2. Apply spatial analysis (buffering, intersection queries, etc.)
# 3. Export subsets to other formats for external tools
# 4. Validate specific maritime features (e.g., navigable areas, obstructions)
# =============================================================================